# This notebook introduces the sweep classes. 

It is important to note that there are convenience functions which allow us to be more succinct when defining sweeps. Please see: 

https://github.com/sohailc/Qcodes/blob/sweep_integration/docs/examples/sweep/convenience_functions.ipynb

In [1]:
import sys 
sys.path.append(r"C:\Users\a-sochat\development\Qcodes_Sohail_Clone\Qcodes")

import numpy as np

import qcodes as qc
from qcodes.instrument.parameter import ManualParameter
from qcodes.sweep.sweep import (
    Nest, Zip, Chain, ParameterSweep, ParameterWrapper, FunctionSweep, FunctionWrapper, measurement, setter
)

C:\Users\a-sochat\AppData\Local\Continuum\Anaconda3\envs\qcodes\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
"""
We will first make a measurement-like class which, instead of saving results to file prints them to standard output
"""

class Printer:
    def __init__(self, sweep_object): 
        self._ind, self._dep = sweep_object.parameter_table.flatten()
        self._symbols_list = sweep_object.parameter_table.symbols_list()
        
    def __enter__(self): 
        header_ind = "\t".join(["{} [{}]".format(*i) for i in self._ind.items()])
        header_dep = "\t".join(["{} [{}]".format(*i) for i in self._dep.items()])
        sep = " | "
        print((header_ind + sep + header_dep).strip(sep))
        
        return self
    
    def __exit__(self, type, value, traceback): 
        pass

    def __call__(self, result):
        print(" " + "\t ".join([str(result[ip]) for ip in self._symbols_list]))

# Lets introduce a basic sweep object 

In [7]:
x = ManualParameter("x", unit="V")
sweep_object = ParameterSweep(x, lambda: [0, 1])

# Lets print the parameter table...
print(sweep_object.parameter_table)

x [V]|


Everything in front of the "|" represents an indendent parameter. We see that the sweep only defines one parameter, which is independent and has a unit "V". Lets sweep this sweep object...

In [6]:
with Printer(sweep_object) as printer:
    for i in sweep_object: 
        printer(i)

x [V]
 0
 1


We have generated a small 1D coordinate layout with size 2 

# How do we make a 2D sweep? 

In [8]:
y = ManualParameter("y", unit="V")

In [9]:
sox = ParameterSweep(x, lambda: [0, 1])
soy = ParameterSweep(y, lambda: [0, 1])
sweep_object = Nest([soy, sox])

# Lets print the parameter table...
print(sweep_object.parameter_table)

y [V],x [V]|


In [10]:
with Printer(sweep_object) as printer:
    for i in sweep_object:  # X is the inner axis : 
        printer(i)

y [V]	x [V]
 0	 0
 0	 1
 1	 0
 1	 1


This represents a 2D layout of 2x2 

# We can extend this to ND

In [11]:
z = ManualParameter("z", unit="V")

In [12]:
sox = ParameterSweep(x, lambda: [0, 1])
soy = ParameterSweep(y, lambda: [0, 1])
soz = ParameterSweep(z, lambda: [0, 1])
sweep_object = Nest([soz, soy, sox])

# Lets print the parameter table...
print(sweep_object.parameter_table)

z [V],y [V],x [V]|


In [13]:
with Printer(sweep_object) as printer:
    for i in sweep_object: 
        printer(i)

z [V]	y [V]	x [V]
 0	 0	 0
 0	 0	 1
 0	 1	 0
 0	 1	 1
 1	 0	 0
 1	 0	 1
 1	 1	 0
 1	 1	 1


We have created a 2x2x2 layout 

# This is how we can perform a measurement  

In [14]:
m = ManualParameter("m", unit="A")
m.get = lambda: x() ** 2 + y()

In [15]:
x(3)
y(1)
sweep_object = ParameterWrapper(m)

print(sweep_object.parameter_table)

|m [A]


Everything after the "|" represents a dependent parameter

In [16]:
with Printer(sweep_object) as printer:
    for i in sweep_object: 
        printer(i)

m [A]
 10


Wrapping a parameter in the "ParameterWrapper" class makes a sweep object which iterates once and returns the "get" value of the parameter. We can use this to create looped measurements. 

In [17]:
sox = ParameterSweep(x, lambda: [0, 1, 2])
soy = ParameterSweep(y, lambda: [0, 1, 3, 4])
meas = ParameterWrapper(m)
sweep_object =  Nest([soy, sox, meas])

print(sweep_object.parameter_table)

y [V],x [V]|m [A]


Nesting sweep objects will combine the parameters table. This is how we keep track of the fact that the sweep defines three parameters, "x", "y" and "m", the latter of which is a dependent parameter which depends on the former two. When we register the sweep object with the measurement class, we now know which ParamSpecs we need to make  

In [18]:
with Printer(sweep_object) as printer:
    for i in  sweep_object: 
        printer(i)

y [V]	x [V] | m [A]
 0	 0	 0
 0	 1	 1
 0	 2	 4
 1	 0	 1
 1	 1	 2
 1	 2	 5
 3	 0	 3
 3	 1	 4
 3	 2	 7
 4	 0	 4
 4	 1	 5
 4	 2	 8


Lets make another example with two dependent parameters....

In [19]:
n = ManualParameter("n", unit="A")
n.get = lambda: x() - y() ** 2 + 16

sox = ParameterSweep(x, lambda: [0, 1, 2])
soy = ParameterSweep(y, lambda: [0, 1, 3, 4])
meas1 = ParameterWrapper(m)
meas2 = ParameterWrapper(n)
sweep_object =  Nest([soy, sox, meas1, meas2])

print(sweep_object.parameter_table)

y [V],x [V]|m [A],n [A]


In [20]:
with Printer(sweep_object) as printer:
    for i in  sweep_object: 
        printer(i)

y [V]	x [V] | m [A]	n [A]
 0	 0	 0	 16
 0	 1	 1	 17
 0	 2	 4	 18
 1	 0	 1	 15
 1	 1	 2	 16
 1	 2	 5	 17
 3	 0	 3	 7
 3	 1	 4	 8
 3	 2	 7	 9
 4	 0	 4	 0
 4	 1	 5	 1
 4	 2	 8	 2


# Introducing chaining

In [21]:
x(4)
sweep_object = Nest([
    soy, 
    Chain([
        meas1, 
        Nest([sox, meas2])
    ])
])

The above is equivalent to...

```python
for y in [0, 1, 3, 4]: 
    meas1()
    for x in [0, 1, 2]: 
        meas2()
```

Note that in the notebook: 
https://github.com/sohailc/Qcodes/blob/sweep_integration/docs/examples/sweep/convenience_functions.ipynb

We see another way we can define the same thing through convenience functions

In [22]:
print(sweep_object.parameter_table)

y [V]|m [A]
y [V],x [V]|n [A]


We see that we have two dependent parameters: "m" and "n". The former depends on "y" only while the latter depends on "x" and "y". This is the reason why sweep_object.parameter_table.table_list is a *list* of dictionaries. Let us view the result of this sweep object  

In [23]:
with Printer(sweep_object) as printer:
    for i in sweep_object: 
        printer(i)

y [V]	x [V] | m [A]	n [A]
 0	 None	 16	 None
 0	 0	 None	 16
 0	 1	 None	 17
 0	 2	 None	 18
 1	 None	 5	 None
 1	 0	 None	 15
 1	 1	 None	 16
 1	 2	 None	 17
 3	 None	 7	 None
 3	 0	 None	 7
 3	 1	 None	 8
 3	 2	 None	 9
 4	 None	 8	 None
 4	 0	 None	 0
 4	 1	 None	 1
 4	 2	 None	 2


# We can use arbitrary functions instead of parameters as measurements

In [24]:
@measurement([("meas3", "H")])
def measurement_function(): 
    return int(np.random.normal(0, 1) * 10) / 10

In [25]:
sweep_object = Nest([ParameterSweep(x, lambda: [0, 1, 2, 3]), FunctionWrapper(measurement_function)])
print(sweep_object.parameter_table)

x [V]|meas3 [H]


In [26]:
with Printer(sweep_object) as printer: 
    for i in sweep_object: 
        printer(i)

x [V] | meas3 [H]
 0	 0.5
 1	 0.5
 2	 -0.3
 3	 1.0


In [27]:
@measurement([("m3", "H"), ("m4", "H")])
def measurement_function(): 
    meas3 = int(np.random.normal(0, 1) * 10) / 10
    meas4 = int(np.random.normal(-5, 1) * 10) / 10
    return meas3, meas4

In [29]:
sweep_object = Nest([ParameterSweep(x, lambda: [0, 1, 2, 3]), FunctionWrapper(measurement_function)])

print(sweep_object.parameter_table)

x [V]|m3 [H],m4 [H]


In [30]:
with Printer(sweep_object) as printer: 
    for i in sweep_object: 
        printer(i)

x [V] | m3 [H]	m4 [H]
 0	 -1.3	 -6.1
 1	 0.3	 -3.9
 2	 1.8	 -4.5
 3	 0.4	 -5.7


# We can also use functions as loop parameters

In [42]:
t = ManualParameter("z")
t.get = lambda: int(np.random.uniform(0, 100))

@setter([("xs", "V")])
def setter1(value): 
    x.set(2 * value)
    return value 

In [43]:
sweep_object = Nest([FunctionSweep(setter1, lambda: [0, 1, 2, 3]), FunctionWrapper(measurement_function)])

with Printer(sweep_object) as printer:
    for i in sweep_object: 
        printer(i)

xs [V] | m3 [H]	m4 [H]
 0	 0.4	 -5.5
 1	 -0.2	 -5.7
 2	 0.8	 -5.9
 3	 -0.9	 -5.1


In [44]:
@setter([("xs", "V"), ("ys", "V")])
def setter2(xv, yv):
    x.set(xv)
    y.set(yv)

In [45]:
sweep_object = Nest([
    FunctionSweep(setter2, lambda: zip([0, 1, 2, 3], [4, 5, 7, 8])), 
    FunctionWrapper(measurement_function)
])

with Printer(sweep_object) as printer:
    for i in sweep_object: 
        printer(i)

xs [V]	ys [V] | m3 [H]	m4 [H]
 0	 4	 -1.4	 -4.9
 1	 5	 -0.5	 -4.9
 2	 7	 0.9	 -6.0
 3	 8	 -0.2	 -5.4


# Finally, the sweep values need not be a list or an numpy array, but also a generating function

In [38]:
def sweep_values(): 
    value = 0.0
    while value < 2.0:
        value = np.sum(np.random.normal(0, 1, (3,)))
        yield value

In [41]:
sweep_object = ParameterSweep(x, sweep_values)

for i in sweep_object: 
    print(x())

0.43508727435185546
-0.9810567871821754
0.5366477121615955
1.3301845095789553
1.1557474282909834
1.5900065293713674
2.0995286704840814


We will loop until the sum of the measurement variables equal 2.0 or more. Since measurement values are three stochstics with a N(0, 1) distribution, the distribution of the sum is N(0, sqrt(3)). The probability of finding 2.0 or more: Z = 2/sqrt(3), which is equal to 12.51%. As the next cell shows, the expectation value of the number of iterations is therefore 8.0 iterations

In [42]:
def f(ni): # The probability of looping exactly ni times 
    p = 0.1251
    return (1 - p)**(ni - 1) * p

np.sum([ni*f(ni) for ni in range(1, 1000)])

7.993605115907274

Lets see if this is correct 

In [43]:
count = 0
N = 10000
for _ in range(N):
    count += len(list(sweep_object))  # unroll the sweep object by recasting it to a list

print(count/N)

7.9593


Yayyy!! :-)

The ability to use generators as values we sweep over gives us the ability to create complex loops with feedback mechanisms. See for example: 

https://github.com/sohailc/Qcodes/blob/sweep_integration/docs/examples/DataSet/sweep/example_measurement_with_sweep.ipynb